In [10]:
import numpy as np
import time
from scipy import io
import sys
sys.path.append("..")
from utils import test_train_split, evaluate_model_torch, subtract_spont, corrcoef, PCA,zscore

class EnsemblePursuitNumpy():
    def __init__(self,n_ensembles,lambd,options_dict):
        self.n_ensembles=n_ensembles
        self.lambd=lambd
        self.options_dict=options_dict

    def zscore(self,X):
        mean_stimuli=np.mean(X.T,axis=0)
        std_stimuli=np.std(X.T,axis=0,ddof=1)+1e-10
        X=np.subtract(X.T,mean_stimuli)
        X=np.divide(X,std_stimuli)
        return X.T

    def calculate_dot_squared(self,C_summed):
        dot_squared=np.clip(C_summed,a_min=0,a_max=None)**2
        return dot_squared

    def calculate_cost_delta(self,C_summed,current_v):
        cost_delta=np.clip(C_summed,a_min=0,a_max=None)**2/(self.sz[1]*(current_v**2).sum())-self.lambd
        return cost_delta

    def mask_dot_squared(self,selected_neurons,dot_squared):
        mask=np.zeros((selected_neurons.shape[0]),dtype=bool)
        mask[selected_neurons==0]=1
        mask[selected_neurons!=0]=0
        masked_dot_squared=mask*dot_squared
        return masked_dot_squared

    def sum_C(self,C_summed_unnorm,C,max_delta_neuron):
        C_summed_unnorm=C_summed_unnorm+C[:,max_delta_neuron]
        return C_summed_unnorm

    def sum_v(self, v, max_delta_neuron, X):
        current_v=v+X[max_delta_neuron,:]
        return current_v

    def fit_one_ensemble(self,X,C):
        #A parameter to account for how many top neurons we sample from. It starts from 1,
        #because we choose the top neuron when possible, e.g. when we can find an ensemble
        # that is larger than min ensemble size. If there is no ensemble with the top neuron
        # we increase the number of neurons to sample from.
        self.n_neurons_for_sampling=1
        n=0
        min_assembly_size=self.options_dict['min_assembly_size']
        #index for switching between top neurons for fitting ensemble when the first neurons
        #doesn't give large enough ensemble
        index=-1
        #A while loop for trying sampling other neurons if the found ensemble size is smaller
        #than threshold.
        while n<min_assembly_size:
            start=time.time()
            seed=self.repeated_seed(C,index)
            end=time.time()
            print('old seed', end-start,seed)
            seed=self.fast_sort(C,index)
            end2=time.time()
            print('new seed',end2-end,seed)
            n=1
            current_v=X[seed,:]
            current_v_unnorm=current_v.copy()
            selected_neurons=np.zeros((X.shape[0]),dtype=bool)
            #Seed current_v
            selected_neurons[seed]=1
            #Fake cost to initiate while loop
            max_cost_delta=1000
            C_summed_unnorm=0
            max_delta_neuron=seed
            while max_cost_delta>0:
                #Add the x corresponding to the max delta neuron to C_sum. Saves computational
                #time.
                #print(n)
                C_summed_unnorm=self.sum_C(C_summed_unnorm,C,max_delta_neuron)
                C_summed=(1./n)*C_summed_unnorm
                dot_squared=self.calculate_dot_squared(C_summed)
                #invert the 0's and 1's in the array which stores which neurons have already
                #been selected into the assembly to use it as a mask
                masked_dot_squared=self.mask_dot_squared(selected_neurons,dot_squared)
                max_delta_neuron=np.argmax(masked_dot_squared)
                cost_delta=self.calculate_cost_delta(C_summed[max_delta_neuron],current_v)
                #print(cost_delta)
                if cost_delta>0:
                    selected_neurons[max_delta_neuron]=1
                    current_v_unnorm= self.sum_v(current_v_unnorm,max_delta_neuron,X)
                    n+=1
                    current_v=(1./n)*current_v_unnorm
                max_cost_delta=cost_delta

            index+=-1
        print('nr of neurons in ensemble',n)
        current_u=np.zeros((X.shape[0],1))
        current_u[selected_neurons,0]=np.clip(C_summed[selected_neurons],a_min=0,a_max=None)/(current_v**2).sum()
        self.U=np.concatenate((self.U,current_u),axis=1)
        self.V=np.concatenate((self.V,current_v.reshape(1,self.sz[1])),axis=0)
        return current_u, current_v, C, selected_neurons
               
        
    def fit_one_ensemble_(self,X,C):
        #A parameter to account for how many top neurons we sample from. It starts from 1,
        #because we choose the top neuron when possible, e.g. when we can find an ensemble
        # that is larger than min ensemble size. If there is no ensemble with the top neuron
        # we increase the number of neurons to sample from.
        self.n_neurons_for_sampling=1
        n=0
        min_assembly_size=self.options_dict['min_assembly_size']
        #index for switching between top neurons for fitting ensemble when the first neurons
        #doesn't give large enough ensemble
        index=-1
        #A while loop for trying sampling other neurons if the found ensemble size is smaller
        #than threshold.
        while n<min_assembly_size:
            start=time.time()
            seed=self.repeated_seed(C,index)
            end=time.time()
            print('old seed', end-start,seed)
            seed=self.fast_sort(C,index)
            end2=time.time()
            print('new seed',end2-end,seed)
            n=1
            current_v=X[seed,:]
            current_v_unnorm=current_v.copy()
            selected_neurons=np.zeros((X.shape[0]),dtype=bool)
            #Seed current_v
            selected_neurons[seed]=1
            #Fake cost to initiate while loop
            max_cost_delta=1000
            C_summed_unnorm=0
            max_delta_neuron=seed
            while max_cost_delta>0:
                #Add the x corresponding to the max delta neuron to C_sum. Saves computational
                #time.
                #print(n)
                C_summed_unnorm=self.sum_C(C_summed_unnorm,C,max_delta_neuron)
                C_summed=(1./n)*C_summed_unnorm
                dot_squared=self.calculate_dot_squared(C_summed)
                #invert the 0's and 1's in the array which stores which neurons have already
                #been selected into the assembly to use it as a mask
                masked_dot_squared=self.mask_dot_squared(selected_neurons,dot_squared)
                max_delta_neuron=np.argmax(masked_dot_squared)
                cost_delta=self.calculate_cost_delta(C_summed[max_delta_neuron],current_v)
                if cost_delta>0:
                    selected_neurons[max_delta_neuron]=1
                    current_v_unnorm= self.sum_v(current_v_unnorm,max_delta_neuron,X)
                    n+=1
                    current_v=(1./n)*current_v_unnorm
                max_cost_delta=cost_delta
            index+=-1
        print('nr of neurons in ensemble',n)
        current_u=np.zeros((X.shape[0],1))
        current_u[selected_neurons,0]=np.clip(C_summed[selected_neurons],a_min=0,a_max=None)/(current_v**2).sum()
        self.U=np.concatenate((self.U,current_u),axis=1)
        self.V=np.concatenate((self.V,current_v.reshape(1,self.sz[1])),axis=0)
        return current_u, current_v, C, selected_neurons

    def repeated_seed(self,C,index):
        nr_neurons_to_av=self.options_dict['seed_neuron_av_nr']
        sorted_similarities=np.sort(C,axis=1)[:,:-1][:,C.shape[0]-nr_neurons_to_av-1:]
        average_similarities=np.mean(sorted_similarities,axis=1)
        top_neurons=np.argsort(average_similarities)
        seed=top_neurons[index]
        return seed

    def sorting_for_seed(self,C):
        '''
        This function sorts the similarity matrix C to find neurons that are most correlated
        to their nr_neurons_to_av neighbors (we average over the neighbors).
        '''
        nr_neurons_to_av=self.options_dict['seed_neuron_av_nr']
        sorted_similarities=np.sort(C,axis=1)[:,:-1][:,self.sz[0]-nr_neurons_to_av-1:]
        average_similarities=np.mean(sorted_similarities,axis=1)
        return seed
    
    def fast_sort(self,C,index):
        nr_neurons_to_av=self.options_dict['seed_neuron_av_nr']
        cutoff=self.sz[0]-nr_neurons_to_av-1
        sorted_similarities_fast=np.partition(C,cutoff,axis=1)[:,cutoff:]
        average_similarities_fast=(np.mean(sorted_similarities_fast,axis=1)*sorted_similarities_fast.shape[1]-np.max(sorted_similarities_fast,axis=1))/(sorted_similarities_fast.shape[1]-1)
        top_neurons=np.argsort(average_similarities_fast)
        seed=top_neurons[index]
        #seed=np.argmax(average_similarities_fast)
        return seed
    
    def update_C(self,X,C,u,v,selected_neurons):
        #selected_neurons=np.nonzero(u)[0]
        cross_term_init=X@(v.T)
        cross_term=np.outer(u[selected_neurons],cross_term_init)
        C[selected_neurons,:]=C[selected_neurons,:]-cross_term
        ixgrid=np.ix_(~selected_neurons,selected_neurons)
        C[ixgrid]=C[ixgrid]-cross_term.T[~selected_neurons,:]
        return C

    def fit_transform(self,X):
        X=self.zscore(X)
        self.sz=X.shape
        self.U=np.zeros((X.shape[0],1))
        self.V=np.zeros((1,X.shape[1]))
        C=X@X.T
        #np.fill_diagonal(C,0)
        for iteration in range(0,self.n_ensembles):
            current_u, current_v, C,selected_neurons=self.fit_one_ensemble(X,C)
            U_V=current_u.reshape(self.sz[0],1)@current_v.reshape(1,self.sz[1])
            X=X-U_V
            C=X@X.T
            C=self.update_C(X,C,current_u,current_v,selected_neurons)
            #np.fill_diagonal(C,0)
            print('ensemble nr', iteration)
            cost=np.mean(X*X)
            print('cost',cost)
        #After fitting arrays discard the zero initialization rows and columns from U and V.
        self.U=self.U[:,1:]
        self.V=self.V[1:,:]
        return self.U, self.V.T

In [2]:
path='/home/maria/Documents/EnsemblePursuit_old/experiments/natimg2800_M170717_MP034_2017-09-11.mat'
data = io.loadmat(path)
resp = data['stim'][0]['resp'][0]
spont =data['stim'][0]['spont'][0]
X=subtract_spont(spont,resp).T

In [11]:
options_dict={'seed_neuron_av_nr':100,'min_assembly_size':8}
ep_np=EnsemblePursuitNumpy(n_ensembles=3,lambd=0.01,options_dict=options_dict)
U,V=ep_np.fit_transform(X)

old seed 5.5862157344818115 1165
new seed 0.8343744277954102 1165
nr of neurons in ensemble 1432
ensemble nr 0
cost 0.9940062256834048
old seed 5.260864019393921 9744
new seed 0.8656191825866699 9744
nr of neurons in ensemble 4981


KeyboardInterrupt: 

In [ ]:
nr_neurons_to_av=self.options_dict['seed_neuron_av_nr']
        cutoff=self.sz[0]-nr_neurons_to_av-1
        sorted_similarities_fast=np.partition(C,cutoff,axis=1)[:,cutoff:]
        average_similarities_fast=(np.mean(sorted_similarities_fast,axis=1)*sorted_similarities_fast.shape[1]-np.max(sorted_similarities_fast,axis=1))/(sorted_similarities_fast.shape[1]-1)
        top_neurons=np.argsort(average_similarities_fast)
        seed=top_neurons[index]
        #seed=np.argmax(average_similarities_fast)
        return seed

In [9]:
options_dict={'seed_neuron_av_nr':100,'min_assembly_size':8}
ep_np=EnsemblePursuitNumpy(n_ensembles=150,lambd=0.01,options_dict=options_dict)
U,V=ep_np.fit_transform(X)

seed 1165
seed fast 1165
nr of neurons in ensemble 1432
ensemble nr 0
cost 0.9940062256834048
seed 1165
seed fast 1165
nr of neurons in ensemble 1432
ensemble nr 1
cost 14641046712.677303


KeyboardInterrupt: 

In [25]:
import torch
import numpy as np
from scipy.sparse.linalg import eigsh

def zscore(X, axis=0):
    mean_X= np.mean(X,axis=axis)
    std_X = np.std(X, axis=axis) + 1e-10
    X -= np.expand_dims(mean_X, axis)
    X /= np.expand_dims(std_X, axis)

    return X

def test_train_split(data,stim):
    unique, counts = np.unique(stim.flatten(), return_counts=True)
    count_dict=dict(zip(unique, counts))

    keys_with_enough_data=[]
    for key in count_dict.keys():
        if count_dict[key]==2:
            keys_with_enough_data.append(key)

    filtered_stims=np.isin(stim.flatten(),keys_with_enough_data)

    #Arrange data so that responses with the same stimulus are adjacent
    z=stim.flatten()[np.where(filtered_stims)[0]]
    sortd=np.argsort(z)
    istim=np.sort(z)
    X=data[filtered_stims,:]
    out=X[sortd,:].copy()

    x_train=out[::2,:]
    y_train=istim[::2]
    x_test=out[1::2,:]
    y_test=istim[1::2]

    return x_train, x_test, y_train, y_test


def stimulus_correlation(V, istim):
    x_train,x_test,y_train,y_test= test_train_split(V,istim)
    cc = np.mean(zscore(x_train) * zscore(x_test), axis=0)
    return cc

def evaluate_model(V,istim):
    x_train,x_test,y_train,y_test= test_train_split(V,istim)

    corr_mat = corr_matrix(x_train.T, x_test.T)
    print(np.mean(np.argmax(corr_mat, axis=0) == np.arange(0,x_train.shape[0],1,int)))


def corr_matrix(x,y):
    '''
    calculate correlation matrix
    '''

    x = x- np.mean(x,axis=0)
    y = y- np.mean(y,axis=0)

    x /= np.std(x,axis=0) + 1e-10
    y /= np.std(y,axis=0) + 1e-10

    c = x.T @ y

    return c


def subtract_spont(spont,resp):
    #print(spont)
    mu = spont.mean(axis=0)
    sd = spont.std(axis=0) + 1e-6
    resp = (resp - mu) / sd
    spont = (spont - mu) / sd
    sv,u = eigsh(spont.T @ spont, k=32)
    resp = resp - (resp @ u) @ u.T
    return resp

In [30]:
import numpy as np
from sklearn.decomposition import PCA
from scipy.stats import skew
import time
from scipy.stats import zscore
import sys

def new_ensemble(X, C, seed_timecourse, lam, discard_first_neuron = False):
    # X are the NT by NN neural activity traces (z-scored)
    # C is the covariance matrix of X
    # seed_timecourse initializes the ensemble
    # lam is the explained variance threshold
    # discard the first neuron in the ensemble (if this was used to seed the pursuit)

    NT, NN = X.shape
    mask_neurons=np.ones((NN,),dtype=bool)

    # compute initial bias
    bias = seed_timecourse @ X
    current_v = seed_timecourse

    # initialize C_summed
    C_summed = bias.flatten()

    # keep track of neuron order
    iorder = np.zeros(NN, 'int32')

    n = 0
    while True:
        # at each iteration, first determine the neuron to be added
        imax = np.argmax(C_summed * mask_neurons)

        # compute norm of ensemble trace
        vnorm = np.sum(current_v**2)

        # compute delta cost function
        cost_delta = np.maximum(0., C_summed[imax])**2 / vnorm

        # if cost/variance explained is less than lam (* n_timepoints) then break
        if cost_delta<lam*NT:
            break

        # zero out freshly added neuron
        mask_neurons[imax] = False

        if n==0 and discard_first_neuron:
            discard_first_neuron = False
            continue

        # add column of C
        C_summed = C_summed + C[:, imax]

        # add column of X
        current_v = current_v + X[:, imax]

        # keep track of neurons in ensembles
        iorder[n] = imax

        n = n+1

    # take only first n neurons
    iorder = iorder[:n]

    return iorder, current_v

def one_round_of_kmeans(V, X, lam=0.01, threshold=True):
    # V are the NT by nK cluster activity traces
    # X are the NT by NN neural activity traces (z-scored)
    # if the threshold is true, neurons only make it into a cluster if their explained variance is above lam
    # this is useful in the last stages when most neurons only have noise left

    NT, nK = V.shape

    # computes projections of neurons onto components
    cc = V.T @ X

    # take the biggest projection component for each neuron
    imax = np.argmax(cc, axis=0)

    # for every neuron, compute max projection
    w = np.max(cc, axis=0)

    # explained variance for each neuron
    amax = np.maximum(0, w)**2/NT

    # initialize total explained variance for each cluster
    vm = np.zeros((nK,))

    # update each cluster in k-means
    for j in range(nK):
        # take all neurons assigned to this cluster
        ix = imax==j
        if threshold:
            ix = np.logical_and(ix, amax>lam)

        # if there are more than 0 neurons assigned
        if np.sum(ix)>0:
            # update the component to be the mean  of assigned neurons
            V[:,j] = X[:, ix] @ w[ix]

            # compute total explained variance for this cluster
            vm[j] = np.sum(amax[ix])

    # re-normalize each column of V separately
    V = V/np.sum(V**2 + 1e-6, axis=0)**.5

    return V, vm

def one_round_of_PCA(V, C):
    # computes projections of neurons onto components
    for t in range(5):
        V = C @ V
        V /= np.sum(V**2)**.5
        V = V.flatten()

    return V

def initialize_kmeans(X, nK, lam):
    # initialize k-means for matrix X (NT by NN)
    # the columns of X should be Z-SCORED

    # initialize k-means centers. THINK HOW TO MAKE THIS DETERMINISTIC
    t0 = time.time()
    model = PCA(n_components=nK, random_state = 101).fit(X.T)
    V = model.components_.T
    V = V * np.sign(skew(V.T @ X,axis=1))
    print('obtained %d PCs in %2.4f seconds'%(nK, time.time()-t0))

    #np.random.seed(101)
    #rperm = np.random.permutation(X.shape[1])
    #V = X[:, rperm[:nK]]
    #V = np.random.randn(X.shape[0], nK)

    # keep V as unit norm vectors
    V /= np.sum(V**2 + 1e-6, axis=0)**.5

    t0 = time.time()

    # 10 iterations is plenty
    for j in range(10):
        # run one round of k-means and update the cluster activities (V)
        V, vm = one_round_of_kmeans(V, X, lam, j>5)

    print('initialized %d clusters with k-means in %2.4f seconds'%(nK, time.time()-t0))

    return V, vm

def initialize_sort(C):
    nr_neurons_to_av=100
    cutoff=C.shape[0]-nr_neurons_to_av-1
    sorted_similarities_fast=np.partition(C,cutoff,axis=1)[:,cutoff:]
    average_similarities_fast=(np.mean(sorted_similarities_fast,axis=1)*sorted_similarities_fast.shape[1]-np.max(sorted_similarities_fast,axis=1))/(sorted_similarities_fast.shape[1]-1)
    #top_neurons=np.argsort(average_similarities_fast)
    #seed=top_neurons[index]
    seed=np.argmax(average_similarities_fast)
    return seed

class EnsemblePursuit():
    def __init__(self,n_components= 100,lam=0.01, n_kmeans = 25):
        self.n_components=n_components
        self.lam=lam
        self.n_kmeans=n_kmeans

    def fit(self,X):
        nK=self.n_components
        lam=self.lam
        n_kmeans = self.n_kmeans

        NT, NN = X.shape

        # z-score along time dimension
        X = zscore(X, axis=0)

        # convert to float64 for numerical precision
        X = np.float64(X)

        # initialize k-means clusters and compute their variance in vm
        V, vm = initialize_kmeans(X, n_kmeans, lam)

        # initialize vectors in ensemble pursuit (Vs)
        vs = np.zeros((NT, nK))

        # initialize U
        U = np.zeros((NN, nK))

        # precompute covariance matrix of neurons
        C = X.T @ X

        # keep track of number of neurons per ensemble
        ns = np.zeros(nK,)

        # time the ensemble pursuit
        t0 = time.time()

        # keep track of neuron order in ensembles

        self.order = []
        self.seed = np.zeros((NT, nK))

        #  outer loop
        for j in range(nK):
            # initialize with "biggest" k-means ensemble (by variance)
            imax = np.argmax(vm)

            # zscore the seed trace
            seed = zscore(V[:, imax])
            seed=initialize_sort(C)

            # fit one ensemble starting from this seed
            iorder, current_v  = new_ensemble(X, C, seed, lam)
            self.order.append(iorder)
            self.seed[:, j] = seed

            # keep track of number of neurons
            ns[j] = len(iorder)

            # normalize current_v to unit norm
            current_v /= np.sum(current_v**2)**.5

            # update column of Vs
            vs[:,j] = current_v

            # projection of each neuron onto this ensemble trace
            w = current_v @ X

            # update weights for neurons in this ensemble
            U[iorder, j] = w[iorder]

            # update activity trace
            X[:, iorder] -= np.outer(current_v, w[iorder])

            # rank one update to C using wtw
            wtw = np.outer(w[iorder],  w)

            # update the columns
            C[:, iorder] -= wtw.T

            # update the rows
            C[iorder, :] -= wtw

            # add back term for the submatrix of neurons in this ensemble
            C[iorder[:, np.newaxis], iorder] += wtw[:, iorder]

            # run one round of k-means because we changed X
            #V, vm = one_round_of_kmeans(V, X, lam)

            if j%25==0 or j == nK-1:
                print('ensemble %d, time %2.2f, nr neurons %d, EV %2.4f'%(j, time.time() - t0, len(iorder), 1-np.mean(X**2)))
        print('average sparsity is %2.4f'%(np.mean(U>1e-5)))

        self.components_ = vs
        self.weights = U
        #self.residual_kmeans = V

        # the fit function has to return the model
        return self

In [27]:
ep=EnsemblePursuit(n_components=150,lam=0.01,n_kmeans=150)
ep.fit(X.T)

obtained 150 PCs in 3.7254 seconds
initialized 150 clusters with k-means in 10.2868 seconds
ensemble 0, time 2.65, nr neurons 1432, EV 0.0058
ensemble 25, time 40.26, nr neurons 253, EV 0.0466
ensemble 50, time 66.43, nr neurons 72, EV 0.0638
ensemble 75, time 88.76, nr neurons 70, EV 0.0752
ensemble 100, time 108.50, nr neurons 37, EV 0.0847
ensemble 125, time 126.65, nr neurons 90, EV 0.0926
ensemble 149, time 142.21, nr neurons 44, EV 0.0992
average sparsity is 0.0184


In [31]:
ep_=EnsemblePursuit(n_components=150,lam=0.01,n_kmeans=150)
ep_.fit(X.T)

obtained 150 PCs in 3.5741 seconds
initialized 150 clusters with k-means in 9.6340 seconds


ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)